<a href="https://colab.research.google.com/github/ccnmagnoo/colabs/blob/main/mhe_app_heavy_uploader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# mhe service app loader
for heavy duty loader, using pandas and firebase firestore.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

#firebase
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
#service as account
cred = credentials.Certificate("/content/drive/MyDrive/MinEnergia/01.MHe/Nac/myappt51-firebase-adminsdk-gvror-71ae723740.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

## Loading data from Google Drive

In [ ]:
from pandas.core.frame import DataFrame
#file loader
path = '/content/drive/MyDrive/MinEnergia/01.MHe/Nac/src/'
#rooms
roomFile = '03-2022-room.csv'
roomSource = path + roomFile
roomPd = pd.read_csv(roomSource).replace({np.nan:None})
#consolidated
cvnFile = '03-2022-cvn.csv'
cvnSource = path + cvnFile
cvnPd = pd.read_csv(cvnSource).replace({np.nan:None})
#status
print('room table:', roomPd.shape, 'beneficiaries table: ',cvnPd.shape)

room table: (200, 9) beneficiaries table:  (13786, 17)


In [ ]:
#Database Collection Reference
admin_uid = 'fa37Q3WqWOUoXCLXyAm0kRwmQaJ2'
roomColl = db.collection('Activity').document(admin_uid).collection('Classroom')
cvnColl = db.collection('Activity').document(admin_uid).collection('Consolidated')
testColl = db.collection('Activity').document(admin_uid).collection('Test')

In [ ]:
##source room Schema
roomPd

,idcal,noTaller,region,comuna,fecha,coorganizador,convocados,op,uuid
0,R001.03,1,3,Chañaral,01-01-2016,Municipalidad De Chañaral,54,CNPam9UmrcejpDj0xIMPxBJAc132,nacQ497BH7o7YyNMu0gO
1,R002.03,2,3,Diego De Almagro,02-01-2016,Municipalidad De Diego De Almagro,288,CNPam9UmrcejpDj0xIMPxBJAc132,naczivOjB0vt11K6uxeU
2,R003.03,3,3,Diego De Almagro,03-01-2016,Municipalidad De Diego De Almagro,36,CNPam9UmrcejpDj0xIMPxBJAc132,nacvEMOSNIiaR9wkdxWg
3,R004.03,4,3,Tierra Amarilla,04-01-2016,Municipalidad De Tierra Amarilla,18,CNPam9UmrcejpDj0xIMPxBJAc132,nacDJD07JcbOQmE97lpb
4,R005.03,5,3,Copiapó,05-01-2016,Municipalidad De Copiapó,90,CNPam9UmrcejpDj0xIMPxBJAc132,nacrNMyU4v6GaLouFa5e
...,...,...,...,...,...,...,...,...,...
195,R197.03,197,3,Chañaral,09-12-2021,Municipalidad De Chañaral,86,CNPam9UmrcejpDj0xIMPxBJAc132,nacI78ThvZHVEx4tybng
196,R198.03,198,3,Copiapó,13-12-2021,Municipalidad De Copiapó,79,CNPam9UmrcejpDj0xIMPxBJAc132,nacaJ4dzhc1Tu0tNH2Dl
197,R199.03,199,3,Copiapó,14-12-2021,Municipalidad De Copiapó,93,CNPam9UmrcejpDj0xIMPxBJAc132,nacuXdoDxOVAPzedy3kP
198,R200.03,200,3,Copiapó,15-12-2021,Municipalidad De Copiapó,89,CNPam9UmrcejpDj0xIMPxBJAc132,nacmReaoNLaqa0y14LeC


In [ ]:
%%sql
select
  count(convocados)
  from roomPd as room



UsageError: Cell magic `%%sql` not found.


In [ ]:
#source consolidated Schema
cvnPd.iloc[0]

noTaller                                 1
region                                   3
nombres                        Carmen Rosa
paterno                           Saavedra
materno                              Neira
comuna                            Chañaral
genero                                   F
fecha                           01-01-2016
coorganizador    Municipalidad De Chañaral
fono                               no-data
email                              no-data
getAge                                  74
getGroup                                70
uuid                  nac78Or2lEnUWUyi8ptA
idcal                              R001.03
rut                              5687368-6
room                  nacQ497BH7o7YyNMu0gO
Name: 0, dtype: object

In [ ]:
from pytz import timezone
#converter row -> db schema
def roomConverter (it):
  d,m,y = it.fecha.split('-')
  itDate:datetime = datetime(int(y),int(m),int(d),tzinfo=timezone("America/Santiago"))
  return {
      "allowedCities":[it.comuna],
      "attendees":[],
      "enrolled":[],
      "cityOnOp":it.comuna,
      "colaborator":it.coorganizador,
      "dateInstance": itDate,
      "idCal": it.idcal,
      "land":{"name":it.comuna,"type":"city"},
      "op":{"cur":it.region,"uuid":it.op},
      "placeActivity":{"date":itDate,"dir":"no-dir","name":"no-data"},
      "placeDispatch":{"date":itDate,"dir":"no-dir","name":"no-data"},
      "uuid":it.uuid,
      "vacancies":it.convocados
  }

def userConverter(it):
    d,m,y = it.fecha.split('-')
    itDate:datetime = datetime(int(y),int(m),int(d),tzinfo=timezone("America/Santiago"))
    return {
        "address":{"city":it.comuna,"dir":"no-dir"},
        "classroom":{"dateInstance":itDate,"idCal":it.idcal,"uuid":it.room},
        "dateSign":itDate,
        "dateUpdate":itDate,
        "email":"no-data",
        "gender":it.genero,
        "name":{"fatherName":it.paterno,"firstName":it.nombres,"motherName":it.materno},
        "phone":None,
        "rut":it.rut,
        "sign":"on-paper",
        "uuid":it.uuid
    }


## Intensive dataBase 😠 Loading

In [ ]:
#room LOAD
for row in roomPd.itertuples():
  doc = roomConverter(row)
  print("loading room: ", row.idcal,doc['dateInstance'])
  docRef = roomColl.document(row.uuid)
  docRef.set(doc,
  merge=True)

loading room:  R001.03 2016-01-01 00:00:00-04:43
loading room:  R002.03 2016-01-02 00:00:00-04:43
loading room:  R003.03 2016-01-03 00:00:00-04:43
loading room:  R004.03 2016-01-04 00:00:00-04:43
loading room:  R005.03 2016-01-05 00:00:00-04:43
loading room:  R006.03 2016-06-07 00:00:00-04:43
loading room:  R007.03 2016-06-17 00:00:00-04:43
loading room:  R008.03 2016-06-18 00:00:00-04:43
loading room:  R009.03 2016-06-22 00:00:00-04:43
loading room:  R010.03 2016-06-23 00:00:00-04:43
loading room:  R011.03 2016-06-24 00:00:00-04:43
loading room:  R012.03 2016-07-27 00:00:00-04:43
loading room:  R013.03 2016-07-28 00:00:00-04:43
loading room:  R014.03 2016-07-29 00:00:00-04:43
loading room:  R015.03 2016-07-30 00:00:00-04:43
loading room:  R016.03 2016-07-31 00:00:00-04:43
loading room:  R017.03 2016-08-01 00:00:00-04:43
loading room:  R018.03 2016-08-01 00:00:00-04:43
loading room:  R019.03 2017-11-22 00:00:00-04:43
loading room:  R020.03 2017-07-14 00:00:00-04:43
loading room:  R021.

Bulk Load Beneficiaries : DANGER

In [ ]:
#beneficiaries BULK LOAD
for row in cvnPd.itertuples():
  print("loading user: ", row.rut,row.fecha,row.idcal, row.uuid)
  docRef = cvnColl.document(row.uuid)
  docRef.set(userConverter(row))

Se truncaron las últimas líneas 5000 del resultado de transmisión.
loading user:  7952010-1 12-11-2018 R091.03 nacZAFodkY3WQD6QDvII
loading user:  10068712-7 12-11-2018 R091.03 nacYIosLlcSAoZKQ7hGF
loading user:  18969145-9 12-11-2018 R091.03 nac38M4w7aOpnBPTQd2x
loading user:  16706518-k 12-11-2018 R091.03 nacAifqseOmx9nSZkQ0p
loading user:  15456534-5 12-11-2018 R091.03 nacrXaDIdTIJfZ6SlB8P
loading user:  9424161-8 12-11-2018 R091.03 nacsjeWS5WUtSL4MvT2S
loading user:  10345944-3 12-11-2018 R091.03 nacMWPq9UUfpm6nV4ScC
loading user:  18845524-7 12-11-2018 R091.03 nacvUnODBO69q3lsgc8a
loading user:  13422762-4 12-11-2018 R091.03 nacwvCAND5KzHTvYsL4Z
loading user:  7647992-5 12-11-2018 R091.03 nacMgt6eIoN6PwCux0Zf
loading user:  9647269-2 12-11-2018 R091.03 nac5bE3bfMIwNWjm5ip3
loading user:  7637939-4 12-11-2018 R091.03 naconPMh37Lv1K0O5Eky
loading user:  16773783-8 12-11-2018 R091.03 nacnFcxw5Ok0yj5wH8Mz
loading user:  7847916-7 12-11-2018 R091.03 nacrIW7DiUEPonsvF1k4
loading user:  